## Homework 2
### Ziyan Hu

### Question 1
Implement a function to convert a non-negative integer num < $10^{12}$ to a string of its English words representation, where the first letter of each word should be in upper case. For example, if the input is num = 1234567, the expected output of the function should be “One Million Two Hundred Thirty Four Thousand Five Hundred Sixty Seven”.
Hint: Wrap up the codes you wrote for Q4 in Assignment 1 as a function and reuse this function.

In [41]:
# Define a function to convert a non-negative integer under 10^12 into words.
def numToWords(num):
    
    # Define lists of words for numbers and tens.
    words = ["", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight",
             "Nine", "Ten", "Eleven", "Twelve", "Thirteen", "Fourteen", "Fifteen", 
             "Sixteen", "Seventeen", "Eighteen", "Nineteen"]
    
    tens = ["", "Ten", "Twenty", "Thirty", "Forty", "Fifty", "Sixty", 
            "Seventy", "Eighty", "Ninety"]
    
    string = ""  # Initialize an empty string to build the result.
    
    # Convert millions part of the number to words.
    if num >= 1000000:
        millions = num // 1000000
        string += numToWords(millions) + " Million "  # Recursively convert millions and append "Million" to the string
        num %= 1000000  # Update num by removing millions part
    
    # Convert thousands part of the number to words.
    if num >= 1000:
        thousands = num // 1000
        string += numToWords(thousands) + " Thousand "  # Recursively convert thousands and append "Thousand" to the string
        num %= 1000  # Update num by removing thousands part
    
    # Convert hundreds part of the number to words.
    if num >= 100:
        hundred = num // 100
        string += words[hundred] + " Hundred "  # Append the word for hundreds and "Hundred" to the string
        num %= 100  # Update num by removing hundreds part
    
    # Convert tens part of the number to words.
    if num >= 20:
        ten = num // 10
        string += tens[ten] + " "  # Append the word for tens
        num %= 10  # Update num by removing tens part
    elif num >= 10:
        string += words[num] + " "  # Append the word for numbers from 10 to 19
        num = 0  # Set num to 0 to indicate it's already handled
    
    # Convert ones part of the number to words.
    if num > 0:
        string += words[num]  # Append the word for ones
    
    return string.strip().title()  # Remove trailing spaces, capitalize the first letter, and return the result

# Test the function with an example input.
print(numToWords(1234567))


One Million Two Hundred Thirty Four Thousand Five Hundred Sixty Seven


## Question 2
There is a gambler holding a given amount of initial fund. The gambler will win 1 dollar in each gamble with a fixed probability and lose $1 otherwise. The gambler will keep gambling until (1) the gambler’s fund becomes zero (lose the game) or (2) the gambler’s fund reaches a pre-specified amount (win the game).
(a) Implement a function that accepts the following inputs: (1) initial fund, (2) probability to win each gamble, (3) the pre-specified amount to stop, and returns a boolean value to indicate whether the gambler eventually wins the game.


In [33]:
import random

def simulate_gamblers_game(initial_fund, win_probability, stop_amount):
    # Validate input: Ensure initial_fund and stop_amount are positive integers and win_probability is a float between 0 and 1.
    if not isinstance(initial_fund, int) or initial_fund <= 0:
        raise ValueError("Initial fund must be a positive integer")
    if not isinstance(stop_amount, int) or stop_amount <= 0:
        raise ValueError("Stop amount must be a positive integer")
    if not isinstance(win_probability, float) or win_probability < 0 or win_probability > 1:
        raise ValueError("Win probability must be a float between 0 and 1")
    
    fund = initial_fund  # Initialize the gambler's fund to the initial fund.
    
    while fund > 0 and fund < stop_amount:
        if random.random() <= win_probability:
            fund += 1  # The gambler wins 1 dollar.
        else:
            fund -= 1  # The gambler loses 1 dollar.
    
    if fund == stop_amount:
        return True  # The gambler wins the game.
    else:
        return False  # The gambler loses the game.

(b) Use the above function to implement another function that accepts the following inputs: (1) initial fund, (2) probability to win each gamble, (3) the pre-specified amount to stop, (4) n total, and returns the percentage the gambler will eventually win the game if the gambler repeats the game (with the other three input arguments) n total times.

In [34]:
def calculate_win_percentage(initial_fund, win_probability, stop_amount, n_total):
    # Validate input: Ensure n_total is a positive integer.
    if not isinstance(n_total, int) or n_total <= 0:
        raise ValueError("n_total must be a positive integer")
    
    wins = 0  # Initialize the number of wins to 0.
    
    for i in range(n_total):
        if simulate_gamblers_game(initial_fund, win_probability, stop_amount):
            wins += 1  # The gambler wins the game.
    
    return (wins / n_total) * 100  # Return the percentage of times the gambler wins the game.

(c) Use the above function to estimate the chance the gambler will eventually win the game with (1) initial fund = 5, (2) probability to win each gamble = 0.1, 0.3, 0.5, 0.7, 0.9, (3) the pre-specified amount to stop = 10, (4) n total = 100000.

In [35]:
initial_fund = 5
stop_amount = 10
n_total = 100000

win_probabilities = [0.1, 0.3, 0.5, 0.7, 0.9]

for win_probability in win_probabilities:
    win_percentage = calculate_win_percentage(initial_fund, win_probability, stop_amount, n_total)
    print(f"With a win probability of {win_probability}, the gambler wins the game {win_percentage:.2f}% of the time.")

With a win probability of 0.1, the gambler wins the game 0.00% of the time.
With a win probability of 0.3, the gambler wins the game 1.45% of the time.
With a win probability of 0.5, the gambler wins the game 49.96% of the time.
With a win probability of 0.7, the gambler wins the game 98.53% of the time.
With a win probability of 0.9, the gambler wins the game 100.00% of the time.


### Question 3
Implement a log-cache decorator satisfying the following requirements. 

a. It accepts an input argument log path, and can record the input arguments and output generated in every function call in a .txt log file specified by log path.

b. It accepts a second input argument max log size, and can delete the oldest record as soon as the number of records in our log exceeds max log size. We want to limit the number of records in our log to (1) avoid using too much disk space and (2) avoid spending too much time on reading & writing the .txt log file.

c. When it finds the input arguments are already in our current log, it will directly return the output we recorded for these input arguments, without calling the function again. This means our log is not only used to record what happened to the function but also serves as a “cache” for the function.

d. This decorator can be used on any function that (1) will never contain characters ‘\n’, ‘\t’ in its input arguments and output and (2) will always return a string as output.


In [36]:
# This function reads a log file from the given path and returns its contents as a dictionary.
def read_log(log_path):
    log = {}  # Initialize an empty dictionary to store log data

    try:
        with open(log_path, 'rt') as f:
            log_str = f.read()  # Read the log file as a string
            for line in log_str.split('\n'):
                key, val = line.split('\t')  # Split each line into key and value
                log[key] = val  # Add key-value pairs to the log dictionary

    except FileNotFoundError:
        print('Will create a new log file at {:}'.format(log_path))

    finally:
        return log  # Return the log dictionary

# This function writes a dictionary to a log file at the specified path.
def write_log(log_path, log):
    log_str = '\n'.join(k + '\t' + v for k, v in log.items())  # Convert the log dictionary to a string
    with open(log_path, 'wt') as f:
        f.write(log_str)  # Write the log string to the log file

# This is a decorator function that implements caching of function results using a log file.
def log_cache(log_path, max_log_size):
    
    # This inner decorator function takes the old_func to be decorated.
    def log_cache_inner(old_func):
        
        # This is the new function that replaces the old_func.
        def new_func(*args, **kwargs):
            log = read_log(log_path)  # Read the log file into a dictionary
            key = str(args) + str(kwargs)  # Create a unique key based on function arguments
            
            if key in log:
                # If the key is found in the log, return the cached result.
                print('A cache found for key "{0:}", so the result "{1:}" is directly returned'.format(key, log[key]))
                return log[key]
            
            # If the key is not found in the log, execute the original function.
            result = old_func(*args, **kwargs)
            
            # Update the log with the new key-value pair.
            log[key] = result
            
            if len(log) > max_log_size:
                # If the log size exceeds the maximum allowed size, remove the oldest entry.
                oldest_key = next(iter(log))
                print('The oldest key-value pair "{0:}-{1:}" is deleted from the log, as the max log size {2:d} is reached'.format(oldest_key, log[oldest_key], max_log_size))
                del log[oldest_key]
            
            write_log(log_path, log)  # Write the updated log to the log file
            
            return result  # Return the result of the original function
            
        return new_func  # Return the new decorated function
        
    return log_cache_inner  # Return the inner decorator function


In [37]:
# Some test cases
import random

@log_cache(log_path='log_repeat_string.txt', max_log_size=5)
def repeat_string(string, n):
    return string * n

_ = repeat_string('a', 4)
_ = repeat_string('aa', 2)
_ = repeat_string('b', 3)
_ = repeat_string('aa', 2)
_ = repeat_string('c', 8)
_ = repeat_string('d', 1)
_ = repeat_string('e', 0)
_ = repeat_string('a', 4)
_ = repeat_string('b', 3)

print()

random.seed(0)
results = []
for _ in range(100):
    string = random.choices('vwxyz', k=1)[0]
    n = random.randint(1, 3)
    result = repeat_string(string, n)
    results.append(result)
print(results)

The oldest key-value pair "('x', 2){}-xx" is deleted from the log, as the max log size 5 is reached
The oldest key-value pair "('z', 3){}-zzz" is deleted from the log, as the max log size 5 is reached
The oldest key-value pair "('w', 3){}-www" is deleted from the log, as the max log size 5 is reached
A cache found for key "('aa', 2){}", so the result "aaaa" is directly returned
The oldest key-value pair "('y', 2){}-yy" is deleted from the log, as the max log size 5 is reached
The oldest key-value pair "('y', 3){}-yyy" is deleted from the log, as the max log size 5 is reached
The oldest key-value pair "('a', 4){}-aaaa" is deleted from the log, as the max log size 5 is reached
The oldest key-value pair "('aa', 2){}-aaaa" is deleted from the log, as the max log size 5 is reached
A cache found for key "('b', 3){}", so the result "bbb" is directly returned

The oldest key-value pair "('b', 3){}-bbb" is deleted from the log, as the max log size 5 is reached
The oldest key-value pair "('c', 8

### Question 4
Read this Wikipedia page to learn BM25, a famous method to estimate the relevance of a document to a given search query.
Implement your own user-defined type BM25 according to the formulas in the section The ranking function, and use it to search relevant documents from the BM25 documents.txt con- sisting of 10 short documents, where your search queries could be “American statistician”, “the professor in mathematics”, etc.
You should instantiate your BM25 object by

bm25 = BM25(path to BM25_documents.txt, k1=1.5, b=0.75)

where it will read the BM25 documents.txt, do appropriate preprocessing (indexing) on the documents and return the object bm25. You should then send queries in the following way:

    scores_and_docs_sorted = bm25.search(‘American statistician’),
   
where the scores and docs sorted it returns is a list of tuples like 

[(bm25_score_1, document_1), ..., (bm25_score_10, document_10)]

with BM25 scores sorted in descending order.

In [38]:
import math
import string
from collections import Counter, defaultdict

class BM25:
    def __init__(self, docs_path, k1, b):
        # Initialize the BM25 instance with document path and tuning parameters k1 and b.

        # Read and preprocess the documents from the given file path.
        with open(docs_path, 'rt') as f:
            docs_list = f.read().split('\n')

        # Create a dictionary to store documents with their IDs, filtering out empty lines.
        self.docs = {doc_id: doc for doc_id, doc in enumerate(docs_list) if doc.strip()}
        self.k1 = k1  # Tuning parameter k1
        self.b = b    # Tuning parameter b

        # Calculate various document statistics.
        self.doc_count = len(self.docs)
        self.doc_lengths = {doc_id: len(self.tokenize(doc)) for doc_id, doc in self.docs.items()}
        self.avgdl = sum(self.doc_lengths.values()) / self.doc_count

        # Calculate word statistics for document frequency and IDF.
        self.word_to_doc_and_freq = self.get_word_to_doc_and_freq()
        self.word_to_idf = self.get_word_to_idf()

    def tokenize(self, text):
        # Tokenize a text by splitting on whitespace, converting to lowercase, and removing punctuation.
        return [word.lower().strip().strip(string.punctuation) for word in text.split()]

    def get_word_to_doc_and_freq(self):
        # Calculate word-to-document frequency mapping using a Counter defaultdict.
        word_to_doc_and_freq = defaultdict(Counter)
        for doc_id, doc in self.docs.items():
            for word in self.tokenize(doc):
                word_to_doc_and_freq[word][doc_id] += 1
        return word_to_doc_and_freq

    def get_word_to_idf(self):
        # Calculate the inverse document frequency (IDF) for each word.
        word_to_idf = {}
        for word, doc_and_freq in self.word_to_doc_and_freq.items():
            word_to_idf[word] = math.log((self.doc_count - len(doc_and_freq) + 0.5) / (len(doc_and_freq) + 0.5))
        return word_to_idf

    def get_score(self, query, doc_id):
        # Calculate the BM25 score for a given query and document.

        score = 0
        for word in self.tokenize(query):
            word_freq_in_doc = self.word_to_doc_and_freq[word][doc_id] if word in self.word_to_doc_and_freq else 0
            idf = self.word_to_idf[word] if word in self.word_to_idf else math.log((self.doc_count + 1) / 0.5)
            doc_length = self.doc_lengths[doc_id]

            # BM25 score formula
            score += idf * word_freq_in_doc * (self.k1 + 1) / (word_freq_in_doc + self.k1 * (1 - self.b + self.b * doc_length / self.avgdl))

        return score

    def search(self, query):
        # Perform a BM25 search for a given query over all documents and return ranked results.

        scores_and_docs = []
        for doc_id, doc in self.docs.items():
            score = self.get_score(query, doc_id)
            scores_and_docs.append((score, doc))

        # Sort the results by score in descending order (higher scores first).
        return sorted(scores_and_docs, reverse=True)

# Create an instance of BM25 with the document path and tuning parameters.
bm25 = BM25('BM25_documents.txt', k1=1.5, b=0.75)


In [39]:
scores_and_docs_sorted = bm25.search('American statistician')
for i, (score, doc) in enumerate(scores_and_docs_sorted, start=1):
    print('------ Position {0:d} - BM25 Score {1:.4f} ------'.format(i, score))
    print(doc, end='\n\n')

------ Position 1 - BM25 Score 1.6425 ------
Bradley Efron is an American statistician. Efron is especially known for proposing the bootstrap resampling technique, which has had a major impact in the field of statistics and virtually every area of statistical application. The bootstrap was one of the first computer-intensive statistical techniques, replacing traditional algebraic derivations with data-based computer simulations.

------ Position 2 - BM25 Score 1.2581 ------
Calyampudi Radhakrishna Rao, commonly known as C. R. Rao, was an Indian-American mathematician and statistician. He was professor emeritus at Pennsylvania State University and Research Professor at the University at Buffalo. Rao was honoured by numerous colloquia, honorary degrees, and festschrifts and was awarded the US National Medal of Science in 2002. The American Statistical Association has described him as "a living legend whose work has influenced not just statistics, but has had far reaching implications for